In [1]:
#%matplotlib inline
%matplotlib widget

In [2]:
# initialization
from rayoptics.environment import *
from rayoptics.optical import analyses
from rayoptics.mpl.analysisfigure import Wavefront, RayFanPlot, RayGeoPSF, AnalysisFigure

from matplotlib import gridspec
import matplotlib.colors as mcolors

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# Create a new model

In [3]:
opm = OpticalModel()
sm = opm.seq_model
osp = opm.optical_spec
pm = opm.parax_model

### Define first order aperture and field for system

In [4]:
pupil_diameter = 40.
pupil_radius = pupil_diameter/2
osp.pupil = PupilSpec(osp, key=['object', 'pupil'], value=pupil_diameter)

# single field on-axis
osp.field_of_view = FieldSpec(osp, key=['object', 'angle'], flds=[0.0])

# wavelength for analysis: 550nm
osp.spectral_region = WvlSpec([(550.0, 1.0)], ref_wl=0)

### object at infinity, i.e. collimated input

In [5]:
sm.gaps[0].thi = 1e+11

### setup aperture stop surface, then offset mirror behind the aperture stop

In [6]:
mirror_offset = srf.DecenterData(DecenterType.REV, y=pupil_radius)
opm.add_dummy_plane(sd=pupil_radius, decenter=mirror_offset)
sm.set_stop();

### add parabolic mirror and enable it to tip, tilt and offset

In [7]:
offset_aperture = srf.Circular(radius=pupil_diameter, y_offset=0.)
#offset_aperture = srf.Circular(radius=pupil_radius, y_offset=pupil_radius)
mirror_perturb = srf.DecenterData(DecenterType.DAR)
opm.add_mirror(power=1/80, cc=-1, t=-80, sd=pupil_diameter,
               decenter=mirror_perturb, clear_apertures=[offset_aperture])

### all of the system data has been entered, update the model

In [8]:
opm.update_model()

In [9]:
sm.list_surfaces()

0 Surface(lbl='Obj', profile=Spherical(c=0.0), interact_mode=transmit)
1 Surface(profile=Spherical(c=0.0), interact_mode=transmit)
2 Surface(profile=Conic(c=-0.00625, cc=-1), interact_mode=reflect)
3 Surface(lbl='Img', profile=Spherical(c=-0.0), interact_mode=transmit)


In [10]:
sm.list_decenters()

0 Gap(t=100000000000.0, medium=Air)
1 Gap(t=0.0, medium=Air)
  'REV': Decenter: array([ 0., 20.,  0.]), Tilt: array([0., 0., 0.])
2 Gap(t=-80, medium=Air)
  'DAR': Decenter: array([0., 0., 0.]), Tilt: array([0., 0., 0.])


# List first order data

In [11]:
fod = osp.parax_data.fod
fod.list_first_order_data()

efl                  80
ffl                 -80
pp1                  -0
bfl                  80
ppk                   0
f/#                   2
m                     0
red           -1.25e+09
obj_dist          1e+11
obj_ang               1
enp_dist             -0
enp_radius           20
na obj            2e-10
n obj                 1
img_dist            -80
img_ht            1.396
exp_dist             -0
exp_radius           20
na img           0.2425
n img                -1
optical invariant       0.3491


# Draw a lens picture

In [12]:
isdark = False
layout_plt = plt.figure(FigureClass=InteractiveLayout, opt_model=opm, refresh_gui=None,do_draw_rays=True,
                 do_paraxial_layout=False, is_dark=isdark).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fld, wvl, foc = osp.lookup_fld_wvl_focus(0)

In [14]:
optical_axis_ray = trace_base(opm, [0., 0.], fld, wvl)
list_ray(optical_axis_ray)

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000            0     0.000000     0.000000     1.000000        1e+11
  1:      0.00000      0.00000            0     0.000000     0.000000     1.000000        -1.25
  2:      0.00000     20.00000        -1.25     0.000000    -0.246154    -0.969231        81.25
  3:      0.00000      0.00000            0     0.000000    -0.246154    -0.969231            0


In [15]:
spot_plt = plt.figure(FigureClass=SpotDiagramFigure, opt_model=opm, scale_type=Fit.User_Scale,
                      user_scale_value=0.1, is_dark=isdark).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
#sm.ifcs[2].decenter.euler[0] = 0.00573  # 0.1 mrad mirror tilt
sm.ifcs[2].decenter.euler[0] = 0.0573  # 1 mrad mirror tilt
opm.update_model()

In [17]:
spot_plt = plt.figure(FigureClass=SpotDiagramFigure, opt_model=opm, scale_type=Fit.User_Scale,
                      user_scale_value=0.025, is_dark=isdark).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
def create_mirror_tilt_dashboard(figs, ray_data_items, foc, tilt_rng, shift_rng):
    alpha_tilt=widgets.FloatSlider(min=-tilt_rng, max=+tilt_rng, step=.05*tilt_rng,
                                description='alpha tilt', value=foc,
                                readout_format='.4f', continuous_update=True)
    x_shift=widgets.FloatSlider(min=-shift_rng, max=+shift_rng, step=.05*shift_rng,
                                description='x shift', value=0.,
                                readout_format='.4f', continuous_update=True)
    y_shift=widgets.FloatSlider(min=-shift_rng, max=+shift_rng, step=.05*shift_rng,
                                description='y shift', value=0.,
                                readout_format='.4f', continuous_update=True)
    sliders = {}
    sliders['alpha tilt'] = alpha_tilt
    sliders['x shift'] = x_shift
    sliders['y shift'] = y_shift

    def slider_update(change):
        sm.ifcs[2].decenter.euler[0] = alpha_tilt.value
        sm.ifcs[2].decenter.dec[0] = x_shift.value
        sm.ifcs[2].decenter.dec[1] = y_shift.value
        opm.update_model()
        optical_axis_ray = trace_base(opm, [0., 0.], fld, wvl)
        
        # apply changes to fans and grids
        for ray_data in ray_data_items:
            ray_data.update_data(build='rebuild')

        # update and plot results
        for fig in figs:
            fig.clf()
            fig.plot()

    alpha_tilt.observe(slider_update, names='value')
    x_shift.observe(slider_update, names='value')
    y_shift.observe(slider_update, names='value')
    
    return defocus, x_shift, y_shift

In [19]:
image_pt, ref_dir, ref_sphere_radius = fld.ref_sphere

In [20]:
image_pt

array([ 0.        , -0.16389492,  0.        ])

In [21]:
fld.ref_sphere[0][1]=0

In [22]:
image_pt

array([0., 0., 0.])

In [23]:
wavefront_plt = plt.figure(FigureClass=WavefrontFigure, opt_model=opm, scale_type=Fit.User_Scale,
                 user_scale_value=50., is_dark=isdark).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
upr_ray = trace_base(opm, [0., 1.], fld, wvl)
list_ray(upr_ray)

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000            0     0.000000     0.000000     1.000000        1e+11
  1:      0.00000     20.00000            0     0.000000     0.000000     1.000000      -5.0413
  2:      0.00000     40.00502      -5.0013     0.000000    -0.471523    -0.881854       85.047
  3:      0.00000     -0.17648            0     0.000000    -0.472404    -0.881382            0


In [25]:
ctr_ray = trace_base(opm, [0., 0.], fld, wvl)
list_ray(ctr_ray)

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000            0     0.000000     0.000000     1.000000        1e+11
  1:      0.00000      0.00000            0     0.000000     0.000000     1.000000      -1.2702
  2:      0.00000     20.00126      -1.2502     0.000000    -0.247138    -0.968980       81.271
  3:      0.00000     -0.16389            0     0.000000    -0.248107    -0.968733            0


In [26]:
lwr_ray = trace_base(opm, [0., -1.], fld, wvl)
list_ray(lwr_ray)

            X            Y            Z           L            M            N               Len
  0:      0.00000     -0.00000            0     0.000000    -0.000000     1.000000        1e+11
  1:      0.00000    -20.00000            0     0.000000    -0.000000     1.000000           -0
  2:      0.00000      0.00000            0     0.000000    -0.001000    -0.999999           80
  3:      0.00000     -0.16001            0     0.000000    -0.002000    -0.999998            0


In [27]:
list_ray(upr_ray, tfrms=sm.gbl_tfrms)

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000       -1e+11     0.000000     0.000000     1.000000        1e+11
  1:      0.00000     20.00000            0     0.000000     0.000000     1.000000      -5.0413
  2:      0.00000     20.00000      -5.0413     0.000000    -0.472404    -0.881382       85.047
  3:      0.00000    -20.17648          -80     0.000000    -0.472404    -0.881382            0


In [28]:
list_ray(ctr_ray, tfrms=sm.gbl_tfrms)

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000       -1e+11     0.000000     0.000000     1.000000        1e+11
  1:      0.00000      0.00000            0     0.000000     0.000000     1.000000      -1.2702
  2:      0.00000      0.00000      -1.2702     0.000000    -0.248107    -0.968733       81.271
  3:      0.00000    -20.16389          -80     0.000000    -0.248107    -0.968733            0


In [29]:
list_ray(lwr_ray, tfrms=sm.gbl_tfrms)

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000       -1e+11     0.000000    -0.000000     1.000000        1e+11
  1:      0.00000    -20.00000            0     0.000000    -0.000000     1.000000           -0
  2:      0.00000    -20.00000            0     0.000000    -0.002000    -0.999998           80
  3:      0.00000    -20.16001          -80     0.000000    -0.002000    -0.999998            0


# Start of dynamic dashboard

In [30]:
img_offset = ctr_ray[0][-1][0]
on_axis_pt = np.array([img_offset[0], img_offset[1]])

ray_xfan = analyses.RayFan(opm, f=fld, wl=wvl, xyfan='x', image_pt_2d=on_axis_pt)
ray_yfan = analyses.RayFan(opm, f=fld, wl=wvl, xyfan='y', image_pt_2d=on_axis_pt)
ray_grid = analyses.RayGrid(opm, f=fld, wl=wvl, image_pt_2d=on_axis_pt)
ray_list = analyses.RayList(opm, num_rays=48, f=fld, wl=wvl, image_pt_2d=on_axis_pt)

In [31]:
xyabr_fan_list = [(ray_xfan, 'dx', dict(num_points=100)),
                  (ray_yfan, 'dy', dict(num_points=100, linestyle='--'))]

In [32]:
opd_fan_list = [(ray_yfan, 'opd', dict(num_points=100, linewidth=2))]

In [33]:
opd = opm.nm_to_sys_units(wvl)
# one wave of defocus
dfoc = opd/(fod.img_na**2/(2*fod.n_img))
qwrt_dfoc = abs(0.25*dfoc)
# one wave of tilt
_, _, ref_sphere_radius = fld.ref_sphere
shft = ref_sphere_radius*opd/fod.exp_radius

In [34]:
def create_focus_dashboard(figs, ray_data_items, foc, dfoc_rng, shift_rng, on_axis_pt):
    defocus=widgets.FloatSlider(min=-dfoc_rng, max=+dfoc_rng, step=.01*dfoc_rng,
                                description='defocus', value=foc,
                                readout_format='.4f', continuous_update=True)
    x_shift=widgets.FloatSlider(min=-shift_rng, max=+shift_rng, step=.01*shift_rng,
                                description='x shift', value=on_axis_pt[0],
                                readout_format='.4f', continuous_update=True)
    y_shift=widgets.FloatSlider(min=-shift_rng, max=+shift_rng, step=.01*shift_rng,
                                description='y shift', value=on_axis_pt[1],
                                readout_format='.4f', continuous_update=True)

    def slider_update(change):
        dfoc_val = defocus.value
        dx = x_shift.value
        dy = y_shift.value

        # apply changes to fans and grids
        for ray_data in ray_data_items:
            ray_data.foc = dfoc_val
            ray_data.image_pt_2d = np.array([dx, dy])
            ray_data.update_data(build='update')

        # update and plot results
        for fig in figs:
            fig.clf()
            fig.plot()

    defocus.observe(slider_update, names='value')
    x_shift.observe(slider_update, names='value')
    y_shift.observe(slider_update, names='value')
    
    return defocus, x_shift, y_shift

In [35]:
fig = plt.figure(FigureClass=AnalysisFigure, data_objs=[ray_grid, ray_xfan, ray_yfan],
                 figsize=[9, 5], tight_layout=True, is_dark=isdark)
gs = gridspec.GridSpec(nrows=8, ncols=13, figure=fig)

Wavefront(fig, gs[:8, :8], ray_grid, user_scale_value=None, do_contours=False, title='Wavefront Map',
          cmap="BrBG_r")
RayFanPlot(fig, gs[:4, 9:], xyabr_fan_list, user_scale_value=None, scale_type='fit',
           yaxis_ticks_position='right', title='Transverse Ray Aberration')
RayFanPlot(fig, gs[4:8, 9:], opd_fan_list, user_scale_value=None, scale_type='fit',
           yaxis_ticks_position='right', title='Wavefront Aberration')
fig.refresh()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
spotfig = plt.figure(FigureClass=AnalysisFigure, data_objs=[ray_grid, ray_xfan, ray_yfan],
                 figsize=[9, 5], tight_layout=True, is_dark=isdark)
gs = gridspec.GridSpec(nrows=2, ncols=2, figure=spotfig)

us = 0.01
scaler = mcolors.Normalize(vmin=0., vmax=80)
gPSF = RayGeoPSF(spotfig, gs[:,:1], ray_list, user_scale_value=us, scale_type='user', dsp_typ='hist2d',
                 title='Geometrical PSF', norm=scaler, vmax=80, cmap='gray')
gPSF = RayGeoPSF(spotfig, gs[:,-1:], ray_list, user_scale_value=us, scale_type='user', dsp_typ='spot',
                 title='Spot Diagram', marker='x')
spotfig.refresh()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
defocus, x_shift, y_shift = create_focus_dashboard([fig, spotfig],
                                                   [ray_grid, ray_xfan, ray_yfan, ray_list],
                                                   osp.defocus.focus_shift, abs(dfoc), 0.2, on_axis_pt)
display(widgets.HBox([defocus, y_shift]))

In [38]:
# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.005

rect_hist2d = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.2, height]

# start with a rectangular Figure
plt.figure(figsize=(8, 8))

ax_hist2d = plt.axes(rect_hist2d)
ax_hist2d.tick_params(direction='in', top=True, right=True)
ax_histx = plt.axes(rect_histx)
ax_histx.tick_params(direction='in', labelbottom=False)
ax_histy = plt.axes(rect_histy)
ax_histy.tick_params(direction='in', labelleft=False)

# the scatter plot:
#ax_hist2d.scatter(x, y)
x_data = ray_list.ray_abr[0]
y_data = ray_list.ray_abr[1]
max_value = max(max(np.nanmax(x_data), -np.nanmin(x_data)),
                max(np.nanmax(y_data), -np.nanmin(y_data)))

bins = 30
edges = np.linspace(-max_value, max_value, num=100)
gamma = 0.3
dset = ax_hist2d.hist2d(*ray_list.ray_abr, bins=edges, norm=mcolors.PowerNorm(gamma))
h, xedges, yedges, qmesh = dset
ax_hist2d.set_facecolor(qmesh.cmap(0))

# now determine nice limits by hand:
binwidth = 0.0025
#lim = np.ceil(np.abs([x, y]).max() / binwidth) * binwidth
#ax_hist2d.set_xlim((-lim, lim))
#ax_hist2d.set_ylim((-lim, lim))

#bins = np.arange(-lim, lim + binwidth, binwidth)
centers = np.linspace(-max_value, max_value, num=99)
ax_histx.plot(centers, np.max(h, axis=1))
ax_histy.plot(np.max(h, axis=0), centers)

#ax_histx.set_xlim(ax_hist2d.get_xlim())
#ax_histy.set_ylim(ax_hist2d.get_ylim())

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
fig_hist = plt.figure(FigureClass=AnalysisFigure, data_objs=[ray_grid, ray_xfan, ray_yfan],
                 figsize=[9, 5], tight_layout=True, is_dark=isdark)
gsh = gridspec.GridSpec(nrows=2, ncols=2, figure=fig_hist)

gPSF = RayGeoPSF(fig_hist, gsh[:,:], ray_list, user_scale_value=.01, scale_type='user', dsp_typ='hist2d',
                 title='Geometrical PSF')
fig_hist.refresh()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …